In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import re
import time
import scrapy
from scrapy.crawler import CrawlerProcess 
import logging

:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named 'service_identity''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


In [2]:
# make month numbers 
def two_digits(x):
    if int(x)<10:
        return str(0) + str(x)
    else:
        return str(x)

In [3]:
#function to convert time to numeric value
def get_min(time_str):
    m, s = time_str.split(':')
    return int(m) + int(s)/60

In [4]:
#starting 5 get a 1, reserves a 0
def starter(x):
    if (x<5):
        return 1
    else:
        return 0
starter_v = np.vectorize(starter)

In [5]:
#function to determine the winner
def winner(x, y):
    home = False
    x['PTS'] = x['PTS'].fillna(0)
    y['PTS'] = y['PTS'].fillna(0)
    home_score = x['PTS'].astype(int).sum()
    away_score = y['PTS'].astype(int).sum()
    dif = home_score-away_score
    if (dif>=0):
        win = x['Team'][0]
        los = y['Team'][0]
        home = True
    else:
        los = x['Team'][0]
        win = y['Team'][0]
    return(win, los, home)

In [6]:
#function to turn scraped data into a dataframe
def create_df(basic_headers, advanced_headers, basic_values, advanced_values):
    df1 = pd.DataFrame(basic_values, columns = basic_headers)
    df1 = df1.replace(to_replace = '[^>]+>', value='', regex=True) # clean data
    df1['MP'] = df1['MP'].replace('', '0:0')
    df1 = df1.replace('', 0) #clean data
    df1['MP'] = df1['MP'].apply(get_min)
    df1['Starter'] = starter_v(df1.index.values) # reports if the player is a starter 
    df2 = pd.DataFrame(advanced_values, columns = advanced_headers)
    df2 = df2.replace(to_replace = '[^>]+>', value='', regex=True)
    df2['MP'] = df2['MP'].replace('', '0:0')
    df2 = df2.replace('', 0) #clean data
    df2['MP'] = df2['MP'].apply(get_min)
    df2['Starter'] = starter_v(df2.index.values) # reports if the player is a starter 
    return pd.merge(df1, df2) #returns one merged df

In [7]:
#fucntion to get the wins and losses of each team
def add_teams_and_records(home, away, home_df, away_df):
    reg = r"^([\w]+ [\w]+ [a-zA-Z]*)"
    home_team = re.findall(reg, home)[0]
    record1 = home[home.find("(")+1:home.find(")")]
    home_win = int(record1[0:record1.find("-")])
    home_loss = int(record1[record1.find("-")+1:])
    away_team = re.findall(reg, away)[0]
    record2 = away[away.find("(")+1:away.find(")")]
    away_win = int(record2[0:record2.find("-")])
    away_loss = int(record2[record2.find("-")+1:])
    home_df['Team'] = home_team
    away_df['Team'] = away_team
    win, los, home = winner(home_df, away_df)
    if(home):
        home_win -= 1
        away_loss -= 1
    else:
        home_loss -= 1
        away_win -= 1
    home_df['Wins'] = home_win
    home_df['Losses'] = home_loss 
    away_df['Wins'] = away_win
    away_df['Losses'] = away_loss 
    return home_df, away_df



In [8]:
#combine individual player stats into team stats
def compress(home):
    home = home.fillna(0)
    home_compressed = pd.Series()
    home_compressed['Team'] = home['Team'][0]
    home_compressed['Wins'] = home['Wins'][0]
    home_compressed['Losses'] = home['Losses'][0]
    home_compressed['Year'] = home['Year'][0]
    home_compressed['Month'] = home['Month'][0]
    home_compressed['Day'] = home['Day'][0]
    home_compressed['Starters MP'] = home['MP'][:5].mean()
    home_compressed['FG'] = home['FG'].astype(int).sum()
    home_compressed['FGA'] = home['FGA'].astype(int).sum()
    home_compressed['FG%'] = home_compressed['FG']/home_compressed['FGA']
    home['3P'] = home['3P'].replace('',0)
    home['3PA'] = home['3PA'].replace('',0)
    home_compressed['3P'] = home['3P'].astype(int).sum()
    home_compressed['3PA'] = home['3PA'].astype(int).sum()
    home_compressed['3P%'] = home_compressed['3P']/home_compressed['3PA']
    home_compressed['FT'] = home['FT'].astype(int).sum()
    home_compressed['FTA'] = home['FTA'].astype(int).sum()
    home_compressed['FT%'] = home_compressed['FT']/home_compressed['FTA']
    home_compressed['ORB'] = home['ORB'].astype(int).sum()
    home_compressed['DRB'] = home['DRB'].astype(int).sum()
    home_compressed['AST'] = home['AST'].astype(int).sum()
    home_compressed['STL'] = home['STL'].astype(int).sum()
    home_compressed['BLK'] = home['BLK'].astype(int).sum()
    home_compressed['TOV'] = home['TOV'].astype(int).sum()
    home_compressed['PF'] = home['PF'].astype(int).sum()
    home_compressed['PTS'] = home['PTS'].replace('', 0, regex=True).astype(int).sum()
    home['+/-'] = home['+/-'].replace('', 0, regex=True)
    home_compressed['+/-'] = home['+/-'].astype(int).sum()
    home_compressed['TS%'] = home['PTS'].astype(int).sum()/(2*(home['FGA'].astype(int).sum()+.44*home['FTA'].astype(int).sum()))
    home_compressed['eFG%'] = (home['FG'].astype(int).sum()+ .5 * home['3P'].astype(int).sum())/home['FGA'].astype(int).sum()
    home_compressed['3PAr'] = home_compressed['3P']/home_compressed['FGA']
    home_compressed['FTr'] = home_compressed['FT']/home_compressed['FGA']
    return home_compressed

In [9]:
#account for team name changes
def update(date, team):
    if date < '200809':
        team = team.replace('cho', 'cha')
        team = team.replace('okc', 'sea')
        team = team.replace('brk', 'njn')
        team = team.replace('nop', 'noh')
    elif date < '201209':
        team = team.replace('cho', 'cha')
        team = team.replace('brk', 'njn')
        team = team.replace('nop', 'noh')
    elif date < '201309':
        team = team.replace('cho', 'cha')
        team = team.replace('nop', 'noh')
    elif date < '201409':
        team = team.replace('cho', 'cha')
    return team

In [10]:
#list of teams abbreviations
teams = ['ATL', 'BKN', 'BOS', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU',
         'IND','LAC','LAL','MEM','MIA','MIL','MIN','NOP','NYK','OKC','ORL','PHI','PHO',
         'POR','SAC','SAS','TOR','UTA','WAS', 'SEA', 'CHA', 'NJN', 'NOH', 'BRK']
#list of teams
long_teams = ['Atlanta Hawks', 'Brooklyn Nets', 'Boston Celtics', 'Charlotte Hornets', 'Chicago Bulls',
              'Cleveland Cavaliers', 'Dallas Mavericks', 'Denver Nuggets', 'Detroit Pistons', 'Golden State Warriors',
              'Houston Rockets', 'Indiana Pacers', 'Los Angeles Clippers', 'Los Angeles Lakers',
              'Memphis Grizzlies', 'Miami Heat', 'Milwaukee Bucks', 'Minnesota Timberwolves',
              'New Orleans Pelicans', 'New York Knicks', 'Oklahoma City Thunder', 'Orlando Magic',
              'Philadelphia 76ers', 'Phoenix Suns', 'Portland Trail Blazers', 'Sacramento Kings',
              'San Antonio Spurs', 'Toronto Raptors', 'Utah Jazz', 'Washington Wizards', 'Seattle SuperSonics', 
              'Charlotte Bobcats', 'New Jersey Nets', 'New Orleans Hornets', 'Brooklyn Nets']
abr = dict(zip(long_teams,teams))
odds_df = pd.read_csv('oddsdf.csv', encoding = "ISO-8859-1") 

In [11]:
#split by home or away
away_teams = odds_df[odds_df.index%2==0]['Team']
home_teams = odds_df[odds_df.index%2==1]['Team']


In [12]:
x = list(set(list(home_teams)+ list(away_teams)))
x.sort()
for y in ['Baltimore', 'East', 'West', 'Pittsburgh', 'Maccabi Haifa Bc','Louisville', 'Miami (FL)', 'Team Giannis', 'USA All Stars',
          'Team LeBron', 'Team Stephen', 'Team USA', 'Team World', 'World All Stars', 'Team LeBron ']:
    x.remove(y)
print(x)

['Atlanta', 'Boston', 'Brooklyn', 'Charlotte', 'Chicago', 'Cleveland', 'Dallas', 'Denver', 'Detroit', 'Golden State', 'Houston', 'Indiana', 'L.A. Clippers', 'L.A. Lakers', 'Memphis', 'Miami', 'Milwaukee', 'Minnesota', 'New Orleans', 'New York', 'Oklahoma City', 'Orlando', 'Philadelphia', 'Phoenix', 'Portland', 'Sacramento', 'San Antonio', 'Toronto', 'Utah', 'Washington']


In [13]:
teams = ['ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU',
         'IND','LAC','LAL','MEM','MIA','MIL','MIN','NOP','NYK','OKC','ORL','PHI','PHO',
         'POR','SAC','SAS','TOR','UTA','WAS']
abr2 = dict(zip(x, teams)) # zip the team names from the odds df and the abreviations for the website url

In [14]:
urltemplate = "https://www.basketball-reference.com/boxscores/{year}{month}{day}0{team}.html"


In [15]:
#only include valid home teams and clean data
home_teams_df = odds_df[odds_df.index%2==1]
home_teams_df = home_teams_df[home_teams_df['Team'].isin(x)]
teams = home_teams_df['Team']
home_teams_df['yearmonth'] = home_teams_df['Year'].astype(str) + home_teams_df['Month'].apply(two_digits)
abreviations = [abr2[x] for x in teams]
abreviations = [update(x[0],x[1]) for x in list(zip(home_teams_df['yearmonth'].tolist(), abreviations))]
home_teams_df['Abr'] = abreviations

In [16]:
#only include valid away teams and clean data
away_teams_df = odds_df[odds_df.index%2==0]
away_teams_df = away_teams_df[away_teams_df['Team'].isin(x)]
teams = away_teams_df['Team']
away_teams_df['yearmonth'] = away_teams_df['Year'].astype(str) + away_teams_df['Month'].apply(two_digits)
abreviations = [abr2[x].lower() for x in teams]
abreviations = [update(x[0], x[1]) for x in list(zip(away_teams_df['yearmonth'].tolist(), abreviations))]
away_teams_df['Abr'] = abreviations

In [17]:
home_teams_df = home_teams_df[['Team', 'Year', 'Month', 'Day', 'Abr']]
home_teams_df['URL'] = 'https://www.basketball-reference.com/boxscores/' + home_teams_df['Year'].astype(str) + home_teams_df['Month'].apply(two_digits) + home_teams_df['Day'].apply(two_digits) + '0' + home_teams_df['Abr'] + '.html'

In [18]:
urls = home_teams_df['URL']
away_team_dict = dict(zip(urls, away_teams_df['Abr']))

In [19]:
#scrapy class to find all game data
#use time to determine function length
t = time.time()
class GameScraper(scrapy.Spider):
    name = "GameData"
    start_urls = urls

    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'FEED_FORMAT':'csv',
        'FEED_URI': 'games_data.csv',
    }
    
    def parse(self, response):
        day = response.url[-11:-9]
        month = response.url[-13:-11]
        year = response.url[-17:-13]
        away = away_team_dict[response.url]
        home = response.url[-8:-5].lower()


        #gets the basic column headers and advanced headers for each team
        away_column_headers = [item[1:-1] for item in response.xpath("//table[contains (@id, 'box_{team}_basic')]/thead".format(team=away)).re('>.+<')][3:]
        away_column_headers_adv = [item[1:-1] for item in response.xpath("//table[contains (@id, 'box_{team}_advanced')]/thead".format(team=away)).re('>.+<')][3:]
        home_column_headers = [item[1:-1] for item in response.xpath("//table[contains (@id, 'box_{team}_basic')]/thead".format(team=home)).re('>.+<')][3:]
        home_column_headers_adv = [item[1:-1] for item in response.xpath("//table[contains (@id, 'box_{team}_advanced')]/thead".format(team=home)).re('>.+<')][3:]
        #get player data
        away_column_data = [item for item in response.xpath("//table[contains (@id, 'box_{team}_basic')]/tbody/tr/td".format(team=away)).re('data-stat[^<]+')]
        away_column_data_adv = [item for item in response.xpath("//table[contains (@id, 'box_{team}_advanced')]/tbody/tr/td".format(team=away)).re('data-stat[^<]+')]
        home_column_data = [item for item in response.xpath("//table[contains (@id, 'box_{team}_basic')]/tbody/tr/td".format(team=home)).re('data-stat[^<]+')]
        home_column_data_adv = [item for item in response.xpath("//table[contains (@id, 'box_{team}_advanced')]/tbody/tr/td".format(team=home)).re('data-stat[^<]+')]
        
        
        
        #reshape data from list into 2d matrix
        away_column_data = np.array([y for y in away_column_data if 'data-stat="reason"' not in y]).reshape((int(len(away_column_data)/20),20))
        away_column_data_adv = np.array([y for y in away_column_data_adv if 'data-stat="reason"' not in y]).reshape((int(len(away_column_data_adv)/15),15))
        
        home_column_data = np.array([y for y in home_column_data if 'data-stat="reason"' not in y]).reshape((int(len(home_column_data)/20),20))
        home_column_data_adv = np.array([y for y in home_column_data_adv if 'data-stat="reason"' not in y]).reshape((int(len(home_column_data_adv)/15),15))
        
        
        away_df = create_df(away_column_headers, away_column_headers_adv, away_column_data, away_column_data_adv)
        home_df = create_df(home_column_headers, home_column_headers_adv, home_column_data, home_column_data_adv)
        
        away_df['Year'] = year
        away_df['Month'] = month
        away_df['Day'] = day
        home_df['Year'] = year
        home_df['Month'] = month
        home_df['Day'] = day
        
        
        home_df, away_df = add_teams_and_records(response.xpath("//div[contains(@id, 'all_box_{team}_basic')]//h2/text()".format(team=home)).get(),
                                                                    response.xpath("//div[contains(@id, 'all_box_{team}_basic')]//h2/text()".format(team=away)).get(), home_df, away_df)
        
        home_df = compress(home_df)
        away_df = compress(away_df)
        
        home_df['Code'] = home + away
        away_df['Code'] = home + away
        home_df['Home'] = 1
        away_df['Home'] = 0
        
    
        
        yield home_df.to_csv('Games/{team}{year}{month}{day}.csv'.format(team = home, day = day, month = month, year = year))
        yield away_df.to_csv('Games/{team}{year}{month}{day}.csv'.format(team = away, day = day, month = month, year = year))
        
process = CrawlerProcess()
process.crawl(GameScraper)
process.start()

2019-04-09 19:14:39 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: scrapybot)
2019-04-09 19:14:39 [scrapy.utils.log] INFO: Versions: lxml 3.8.0.0, libxml2 2.9.9, cssselect 1.0.1, parsel 1.5.1, w3lib 1.17.0, Twisted 17.5.0, Python 3.6.8 |Anaconda custom (x86_64)| (default, Dec 29 2018, 19:04:46) - [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1b  26 Feb 2019), cryptography 2.6.1, Platform Darwin-17.6.0-x86_64-i386-64bit
2019-04-09 19:14:39 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'csv', 'FEED_URI': 'games_data.csv', 'LOG_LEVEL': 30}
2019-04-09 20:04:42 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.basketball-reference.com/boxscores/201102130MEM.html> (referer: None)
Traceback (most recent call last):
  File "/Users/ericweltz/anaconda3/lib/python3.6/site-packages/pandas/core/internals.py", line 4294, in create_block_manager_from_blocks
    placement=slice(0, len(axes[0])))]
  File "/Users/ericwel

In [20]:
print(time.time()-t)

8363.233618736267
